# Transformations and Actions

## Introduction to the Data

In a previous lesson, we touched briefly on transformations and actions, and how these two methods affect the execution of code. In this lesson, we'll dive deeper into how those mechanisms work, and explore a wider range of the functions built into the [Spark core](http://spark.apache.org/docs/latest/api/python/pyspark.html).<br>

The file `hamlet.txt` contains the entire text of [Shakespeare's play Hamlet](https://en.wikipedia.org/wiki/Hamlet). Shakespeare is well-known for his unique writing style and arguably one of the most influential writers in history. Hamlet is one of his most popular plays.<br>

Let's perform some text analysis on it. The file is in pure text format, though, and not ready for analysis. Before we can proceed, we'll have to clean up and reformat the data.

* Read the text file into an RDD named `raw_hamlet` using the `textFile()` method from SparkContext (this object instantiates to `sc` on our end).
* Display the first five elements of the RDD.

In [1]:
import pyspark

In [ ]:
sc = pyspark.SparkContext()

In [3]:
raw_hamlet = sc.textFile("data/hamlet.txt")
raw_hamlet.take(5)

['hamlet@0\t\tHAMLET',
 'hamlet@8',
 'hamlet@9',
 'hamlet@10\t\tDRAMATIS PERSONAE',
 'hamlet@29']

## The Map Method

The text file uses the tab character (`\t`) as a delimiter. We'll need to split the file on the tab delimiter and convert the results into an RDD that's more manageable.

* Use the `map` method to convert:

```python
    ['hamlet@0\t\tHAMLET',
    'hamlet@8',
    'hamlet@9',
    'hamlet@10\t\tDRAMATIS PERSONAE',
    'hamlet@29']
```

to:

```python
    [['hamlet@0', '', 'HAMLET'],
     ['hamlet@8'],
     ['hamlet@9'],
     ['hamlet@10', '', 'DRAMATIS PERSONAE'],
     ['hamlet@29']]
```

Name the resulting RDD `split_hamlet`.

In [4]:
split_hamlet = raw_hamlet.map(lambda x: x.split('\t'))
split_hamlet.take(5)

[['hamlet@0', '', 'HAMLET'],
 ['hamlet@8'],
 ['hamlet@9'],
 ['hamlet@10', '', 'DRAMATIS PERSONAE'],
 ['hamlet@29']]

## Beyond Lambda Functions

Lambda functions are great for writing quick functions we can pass into PySpark methods with simple logic. They fall short when we need to write more customized logic, though. Thankfully, PySpark lets us define a function in Python first, then pass it in. Any function that returns a sequence of data in PySpark (versus a guaranteed Boolean value, like `filter()` requires) must use a `yield` statement to specify the values that should be pulled later.<br>

If you're unfamiliar with the `yield` statement in Python, read this excellent [Stack Overflow answer](http://stackoverflow.com/a/231855) on the topic. To summarize, `yield` is a Python technique that allows the interpreter to generate data on the fly and pull it when necessary, instead of storing it to memory immediately. Because of its unique architecture, Spark takes advantage of this technique to reduce overhead and improve the speed of computations.<br>

Spark runs the named function on every element in the RDD and restricts it in scope. Each instance of the function only has access to the object(s) you pass into the function, and the Python libraries available in your environment. If you try to refer to variables outside the scope of the function or import libraries, those actions may cause the computation to crash. That's because Spark compiles the function's code to Java to run on the RDD objects (which are also in Java).<br>

Finally, not all functions require us to use `yield`; only the ones that generate a custom sequence of data do. For `map()` or `filter()`, we use return to return a value for every single element in the RDD we're running the functions on.

## The FlatMap Method

In the following code cell, we'll use the `flatMap()` method with the named function `hamlet_speaks` to check whether a line in the play contains the text `HAMLET` in all caps (indicating that Hamlet spoke). `flatMap()` is different than `map()` because it doesn't require an output for every element in the RDD. The `flatMap()` method is useful whenever we want to generate a sequence of values from an RDD.<br>

In this case, we want an RDD object that contains tuples of the unique line IDs and the text "hamlet speaketh!," but **only for the elements in the RDD that have "HAMLET" in one of the values**. We can't use the `map()` method for this because it requires a return value for every element in the RDD.<br>

We want each element in the resulting RDD to have the following format:

1. The first value should be the unique line ID (e.g.`'hamlet@0'`) , which is the first value in each of the elements in the `split_hamlet` RDD.
2. The second value should be the string "hamlet speaketh!"

In [5]:
def hamlet_speaks(line):
    id = line[0]
    speaketh = False
    
    if "HAMLET" in line:
        speaketh = True
    
    if speaketh:
        yield id,"hamlet speaketh!"

hamlet_spoken = split_hamlet.flatMap(lambda x: hamlet_speaks(x))
hamlet_spoken.take(10)

[('hamlet@0', 'hamlet speaketh!'),
 ('hamlet@75', 'hamlet speaketh!'),
 ('hamlet@1004', 'hamlet speaketh!'),
 ('hamlet@9144', 'hamlet speaketh!'),
 ('hamlet@12313', 'hamlet speaketh!'),
 ('hamlet@12434', 'hamlet speaketh!'),
 ('hamlet@12760', 'hamlet speaketh!'),
 ('hamlet@12858', 'hamlet speaketh!'),
 ('hamlet@14821', 'hamlet speaketh!'),
 ('hamlet@15261', 'hamlet speaketh!')]

## Filter Using a Named Function

`hamlet_spoken` now contains the line numbers for the lines where Hamlet spoke. While this is handy, we don't have the full line anymore. Instead, let's use a `filter()` with a named function to extract the original lines where Hamlet spoke. The functions we pass into `filter()` **must** return values, which will be either `True` or `False`.

* Write a named function `filter_hamlet_speaks` to pass into `filter()`. Apply it to `split_hamlet` to return an RDD with the elements containing the word `HAMLET`.
  * Assign the resulting RDD to `hamlet_spoken_lines`.

In [6]:
def filter_hamlet_speaks(line):
    if 'HAMLET' in line:
        return True
    return False

hamlet_spoken_lines = split_hamlet.filter(lambda line: filter_hamlet_speaks(line))
hamlet_spoken_lines.take(5)

[['hamlet@0', '', 'HAMLET'],
 ['hamlet@75', 'HAMLET', 'son to the late, and nephew to the present king.'],
 ['hamlet@1004', '', 'HAMLET'],
 ['hamlet@9144', '', 'HAMLET'],
 ['hamlet@12313',
  'HAMLET',
  '[Aside]  A little more than kin, and less than kind.']]

## Actions

As we've discussed before, Spark has two kinds of methods, transformations and actions. While we've explored some of the transformations, we haven't used any actions other than `take()`.<br>

Whenever we use an action method, Spark forces the evaluation of lazy code. If we only chain together transformation methods and print the resulting RDD object, we'll see the type of RDD (e.g. a PythonRDD or PipelinedRDD object), but not the elements within it. That's because the computation hasn't actually happened yet.<br>

Even though Spark simplifies chaining lots of transformations together, it's good practice to use actions to observe the intermediate RDD objects between those transformations. This will let you know whether your transformations are working the way you expect them to.

#### Count()
The `count()` method returns the number of elements in an RDD. `count()` is useful when we want to make sure the result of a transformation contains the right number of elements. For example, if we know there should be an element in the resulting RDD for every element in the initial RDD, we can compare the counts of both to ensure they match.<br>

To get the number of elements in the RDD `hamlet_spoken_lines`, run `.count()` on it:

```python
hamlet_spoken_lines.count()
```

#### Collect()
We've used `take()` to preview the first few elements of an RDD, similar to the way we've use `head()` in pandas. But what about returning all of the elements in a collection? We need to do this to write an RDD to a CSV, for example. It's also useful for running some basic Python code over a collection without going through PySpark.<br>

Running `.collect()` on an RDD returns **a list representation** of it. To get a list of all the elements in hamlet_spoken_lines, for example, we would write:

```python
hamlet_spoken_lines.collect()
```

* Compute the number of elements in `hamlet_spoken_lines`, and assign the result to the variable named `spoken_count`.
* Grab the 101st element in `hamlet_spoken_lines` (which has the list index 100), and assign that list to `spoken_101`.

In [ ]:
spoken_count = 0
spoken_101 = list()
spoken_count = hamlet_spoken_lines.count()
spoken_collect = hamlet_spoken_lines.collect()
spoken_101 = spoken_collect[100]

## Next Steps

While we've done some initial cleanup of the Hamlet data set, we hope you have a better idea of how to use PySpark to transform it into a format that's better for data analysis. We also learned how to use actions to explore an RDD before chaining another transformation to it.<br>

The next mission is a [challenge](https://www.dataquest.io/blog/projects-challenges-tracks/) that will test your understanding of Spark, transformations, actions, lambda functions, and the MapReduce paradigm in general. After that challenge, we'll explore Spark DataFrames and how to analyze data using all the techniques we've learned.<br>

If you'd like to learn how to install PySpark and integrate it with IPython Notebook, [this wonderful blog post](http://ramhiser.com/2015/02/01/configuring-ipython-notebook-support-for-pyspark/) will walk you through the steps. As always, if you have feedback on this mission or Dataquest in general, please reach out to us at `hello@dataquest.io`. If you're not in our [Slack community](https://dscommunity.slack.com/) already, head over to `https://www.dataquest.io/chat`.

In [7]:
sc.stop()